In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary modules
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64
import pandas as pd

# Import AnimalShelter class from ProjectOne
from ProjectOne import AnimalShelter

# Database credentials
username = "aacuser"
password = "SNHU1234"

# Connect to MongoDB database via CRUD module
db = AnimalShelter(username, password)

# Function to fetch all data
def fetch_data(query={}):
    records = db.read(query)
    if not records:
        return pd.DataFrame()  # Return empty DataFrame if no records
    df = pd.DataFrame.from_records(records)
    
    # Drop MongoDB's '_id' column if it exists
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
    
    return df

# Load initial data
df = fetch_data()

# Setup Dash app
app = JupyterDash(__name__)

# Load and encode the logo image
image_filename = 'Grazioso_Salvare_Logo.png'  # Ensure correct filename
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Bobby Eigenfeld'))),
    html.Hr(),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})),
    html.Hr(),
    
    # Filtering options
    html.Div([
        html.Label("Filter by Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster Tracking', 'value': 'Disaster/Individual Tracking'}
            ],
            value='all',
            inline=True
        )
    ]),
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        filter_action='native',
        sort_action='native',
        row_selectable='single'
    ),
    
    html.Br(),
    html.Hr(),
    
    # Graphs and Map
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Callbacks for Interactivity
#############################################

# Update DataTable based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    query = {}  # Default to no filter (returns all data)

    # Map the rescue type to corresponding breeds
    rescue_mapping = {
        "Water Rescue": {"breed": {"$in": ["Labrador Retriever Mix", "Newfoundland", "Chesapeake Bay Retriever"]}},
        "Mountain/Wilderness Rescue": {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Bernese Mountain Dog"]}},
        "Disaster/Individual Tracking": {"breed": {"$in": ["Bloodhound", "Doberman Pinscher", "Golden Retriever"]}}
    }

    # Apply filter if not "all"
    if filter_type != "all":
        query = rescue_mapping.get(filter_type, {})

    df_filtered = fetch_data(query)
    return df_filtered.to_dict('records')

# Update Graphs
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    if not viewData:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if 'breed' in dff.columns and not dff.empty:
        fig = px.pie(dff, names='breed', title='Rescue Animal Distribution')
        return [dcc.Graph(figure=fig)]
    
    return []

# Update Map Based on Selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if not viewData or index is None or len(index) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    # Ensure latitude and longitude columns exist
    if "location_lat" in dff.columns and "location_long" in dff.columns:
        lat, lon = dff.iloc[row]["location_lat"], dff.iloc[row]["location_long"]
    else:
        lat, lon = 30.75, -97.48  # Default location

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]["breed"]),
                dl.Popup([
                    html.H1("Animal Details"),
                    html.P(f"Breed: {dff.iloc[row]['breed']}"),
                    html.P(f"Age: {dff.iloc[row]['age_upon_outcome_in_weeks']} weeks")
                ])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True)


Original snake data:
{'_id': ObjectId('67bbfe81ad10d90b31b5f76f'), 'animal_id': 'A987147', 'animal_type': 'Snake', 'breed': 'Cobra', 'color': 'Brown'}
Updated snake data:
{'_id': ObjectId('67bbfe81ad10d90b31b5f76f'), 'animal_id': 'A987148', 'animal_type': 'Snake', 'breed': 'Cobra', 'color': 'Brown'}
1 document(s) deleted
Dash app running on http://127.0.0.1:26501/
